# Healthcare Operational Trend Analysis (Simulated Lab Data)
Load data, compute risk score, and visualize trends.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../data/simulated_lab_data.csv')
df['collection_date'] = pd.to_datetime(df['collection_date'])
df.head()

In [ ]:
def risk_score(row):
    score = 0
    if row.WBC > 20: score += 2
    elif row.WBC > 11: score += 1
    if row.HGB < 7: score += 2
    elif row.HGB < row.HGB_low: score += 1
    if row.Lactate >= 2.2: score += 2
    elif row.Lactate >= 1.8: score += 1
    if row.CRP >= 50: score += 2
    elif row.CRP >= 10: score += 1
    if row.PLT < 50: score += 2
    elif row.PLT < 150: score += 1
    return int(score)

df['risk_score'] = df.apply(risk_score, axis=1)
df['risk_level'] = np.select([df.risk_score>=6, df.risk_score>=3], ['High','Moderate'], default='Low')
df[['patient_id','collection_date','WBC','HGB','PLT','CRP','Lactate','risk_score','risk_level']].head()

In [ ]:
plt.figure()
df['risk_score'].value_counts().sort_index().plot(kind='bar')
plt.title('Risk Score Distribution')
plt.xlabel('Risk score')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
pt = df.groupby('patient_id')['risk_score'].max().sort_values(ascending=False).index[0]
g = df[df.patient_id==pt].sort_values('collection_date')
plt.figure()
plt.plot(g['collection_date'], g['WBC'], marker='o')
plt.axhline(float(g['WBC_high'].iloc[0]), linestyle='--')
plt.title(f'WBC Trend Over Time ({pt})')
plt.xlabel('Collection date')
plt.ylabel('WBC')
plt.tight_layout()
plt.show()